Data Preprocessing

##  Variable Definitions & Formulas

| Variable              | Definition / Formula |
|-----------------------|----------------------|
| `return`              | Daily stock return |
| `expected_return`     | Estimated return from estimation window:  $\bar{R}_i = \frac{1}{T} \sum_{t=-90}^{-30} R_{i,t}$ |
| `abnormal_return` (AR)| $AR_{i,t} = R_{i,t} - \bar{R}_i$ |
| `CAR_pre`             | $\sum_{t=-10}^{-1} AR_{i,t}$ |
| `CAR_post`            | $\sum_{t=0}^{10} AR_{i,t}$ |
| `delta_CAR`           | $CAR_{\text{post}} - CAR_{\text{pre}}$ |
| `CAR` (custom window) | $\sum_{t=a}^{b} AR_{i,t}$ |
| `event_time`          | $t = \text{stock\_date} - \text{event\_date}$ |
| `post`                | Binary indicator: $1$ if $t \geq 0$, otherwise $0$ |
| `treatment`           | 1 if firm is hardware-dependent, 0 otherwise |
| `treatment × post`    | Interaction term: $treatment \times post$ |
| `firm_FE`             | Firm fixed effect (declared in model setup) |
| `event_FE`            | Event fixed effect (declared in model setup) |




In [1]:
import yfinance as yf
import pandas as pd


def daily_return_price(ticker, start_date, end_date, treatment_flag):
    """
    Fetches historical price and computes daily return for a given ticker.

    Parameters:
        ticker (str): Stock symbol, e.g., 'NVDA'
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        treatment_flag (int): 1 if hardware-dependent (treatment), 0 if control

    Returns:
        DataFrame: Columns = ['Date', 'ticker', 'Close', 'Return', 'Treatment']
    """
    stock = yf.Ticker(ticker)
    price_df = stock.history(start=start_date, end=end_date)

    df = price_df.copy()
    df["Return"] = df["Close"].pct_change()
    df["ticker"] = ticker
    df["Treatment"] = treatment_flag
    df = df.reset_index()  # turn 'Date' from index to column

    return df[["Date", "ticker", "Close", "Return", "Treatment"]]

In [2]:
# Treatment Group
NVDA_df = daily_return_price("NVDA", "2020-12-31", "2024-12-31", 1)  # NVIDIA
AMD_df = daily_return_price("AMD", "2020-12-31", "2024-12-31", 1)  # AMD
INTC_df = daily_return_price("INTC", "2020-12-31", "2024-12-31", 1)  # Intel

# Control Group 1: SaaS/software platform
PLTR_df = daily_return_price("PLTR", "2020-12-31", "2024-12-31", 0)  # Palantir
AI_df = daily_return_price("AI", "2020-12-31", "2024-12-31", 0)  # C3.ai
SOUN_df = daily_return_price("SOUN", "2020-12-31", "2024-12-31", 0)  # SoundHound AI

# Control Group 2:  Software/cloud service companies
CRM_df = daily_return_price("CRM", "2020-12-31", "2024-12-31", 0)  # Salesforce
ORCL_df = daily_return_price("ORCL", "2020-12-31", "2024-12-31", 0)  # Oracle
ADBE_df = daily_return_price("ADBE", "2020-12-31", "2024-12-31", 0)  # Adobe

# Market Data
Market_df = daily_return_price("^GSPC", "2020-12-31", "2024-12-31", 0)  # S&P500
Market_df["ticker"] = "SP500"

In [3]:
def compute_expected_return_rolling(df, estimation_start=-90, estimation_end=-30):
    """
    For each row, compute expected return by averaging the returns of the same ticker
    in the range [Date + estimation_start, Date + estimation_end].

    Parameters:
        df (DataFrame): Must include ['Date', 'ticker', 'Return']
        estimation_start (int): Start of look-back window (e.g., -90)
        estimation_end (int): End of look-back window (e.g., -30)

    Returns:
        DataFrame with 'expected_return' and 'abnormal_return'
    """
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(["ticker", "Date"])

    # Multi-index for efficient slicing
    df_indexed = df.set_index(["ticker", "Date"]).sort_index()

    result = []

    for idx, row in df.iterrows():
        ticker = row["ticker"]
        date = row["Date"]

        start_date = date + pd.Timedelta(days=estimation_start)
        end_date = date + pd.Timedelta(days=estimation_end)

        try:
            history_returns = df_indexed.loc[ticker].loc[start_date:end_date]["Return"]
            expected = history_returns.mean()
        except:
            expected = None

        row["expected_return"] = expected
        row["abnormal_return"] = (
            row["Return"] - expected if pd.notnull(expected) else pd.NA
        )
        result.append(row)

    return pd.DataFrame(result)

In [4]:
# Apply to each stock
NVDA_df = compute_expected_return_rolling(NVDA_df)
AMD_df = compute_expected_return_rolling(AMD_df)
INTC_df = compute_expected_return_rolling(INTC_df)

PLTR_df = compute_expected_return_rolling(PLTR_df)
AI_df = compute_expected_return_rolling(AI_df)
SOUN_df = compute_expected_return_rolling(SOUN_df)

CRM_df = compute_expected_return_rolling(CRM_df)
ORCL_df = compute_expected_return_rolling(ORCL_df)
ADBE_df = compute_expected_return_rolling(ADBE_df)

Market_df = compute_expected_return_rolling(Market_df)

In [5]:
# Main Analysis
main_df = pd.concat(
    [NVDA_df, AMD_df, INTC_df, PLTR_df, AI_df, SOUN_df, Market_df], ignore_index=True
)

# Robustness test
robust_df = pd.concat(
    [NVDA_df, AMD_df, INTC_df, CRM_df, ORCL_df, ADBE_df, Market_df], ignore_index=True
)

In [6]:
event_dict = {
    "2022_export_control": pd.to_datetime("2022-10-07"),
    "2023_investment_ban": pd.to_datetime("2023-08-09"),
    "2023_export_expansion": pd.to_datetime("2023-10-17"),
    "2024_export_tightening": pd.to_datetime("2024-03-29"),
    "2024_tariff_increase": pd.to_datetime("2024-05-14"),
    "2024_investment_finalization": pd.to_datetime("2024-10-28"),
}

In [7]:
def tag_event_info_all_rows(df, event_dict, window=10):
    """
    Tag event_id, event_date, and event_time for rows within ±window of any event,
    while keeping all original rows (unmatched rows get NaN).

    Parameters:
        df (DataFrame): Must include ['Date', 'ticker', ...]
        event_dict (dict): {'event_id': pd.Timestamp('YYYY-MM-DD')}
        window (int): Matching window in days (default ±10)

    Returns:
        DataFrame: Same shape as input, with additional columns:
                   ['event_id', 'event_date', 'event_time']
    """
    df = df.copy()
    df["Date"] = df["Date"].dt.tz_localize(None)

    tag_list = []

    for event_id, event_date in event_dict.items():
        temp = df.copy()
        temp["event_time_candidate"] = (temp["Date"] - event_date).dt.days
        temp["event_id_candidate"] = event_id
        temp["event_date_candidate"] = event_date

        # Keep only matches within window
        temp = temp[
            (temp["event_time_candidate"] >= -window)
            & (temp["event_time_candidate"] <= window)
        ]

        tag_list.append(
            temp[
                [
                    "Date",
                    "ticker",
                    "event_id_candidate",
                    "event_date_candidate",
                    "event_time_candidate",
                ]
            ]
        )

    # Combine all matches
    tag_df = pd.concat(tag_list, ignore_index=True)

    # Merge event info into original df
    df = df.merge(tag_df, on=["Date", "ticker"], how="left")

    # Rename columns
    df.rename(
        columns={
            "event_id_candidate": "event_id",
            "event_date_candidate": "event_date",
            "event_time_candidate": "event_time",
        },
        inplace=True,
    )

    return df

In [8]:
tagged_df = tag_event_info_all_rows(main_df, event_dict, window=10)
tagged_df

,Date,ticker,Close,Return,Treatment,expected_return,abnormal_return,event_id,event_date,event_time
0,2020-12-31,NVDA,13.020754,NaN,1,NaN,<NA>,NaN,NaT,NaN
1,2021-01-04,NVDA,13.079099,0.004481,1,NaN,<NA>,NaN,NaT,NaN
2,2021-01-05,NVDA,13.369586,0.022210,1,NaN,<NA>,NaN,NaT,NaN
3,2021-01-06,NVDA,12.581409,-0.058953,1,NaN,<NA>,NaN,NaT,NaN
4,2021-01-07,NVDA,13.308994,0.057830,1,NaN,<NA>,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...
6697,2024-12-23,SP500,5974.069824,0.007287,0,0.000966,0.006321,NaN,NaT,NaN
6698,2024-12-24,SP500,6040.040039,0.011043,0,0.001033,0.010009,NaN,NaT,NaN
6699,2024-12-26,SP500,6037.589844,-0.000406,0,0.001175,-0.001581,NaN,NaT,NaN
6700,2024-12-27,SP500,5970.839844,-0.011056,0,0.001060,-0.012115,NaN,NaT,NaN


In [9]:
def compute_car_pre_post_delta_full(df, pre_window=(-10, -1), post_window=(0, 10)):
    """
    Compute CAR_pre, CAR_post, delta_CAR, and post indicator for all rows,
    without filtering any row. Only rows in [–10, +10] contribute to CAR computation,
    but all rows are kept.

    Parameters:
        df (DataFrame): Must include ['Date', 'ticker', 'event_date', 'abnormal_return']
        pre_window (tuple): (start_day, end_day) for CAR_pre (e.g., (-10, -1))
        post_window (tuple): for CAR_post (e.g., (0, 10))

    Returns:
        df (DataFrame): original df with added columns: ['event_time', 'CAR_pre', 'CAR_post', 'delta_CAR', 'post']
    """
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df["event_date"] = pd.to_datetime(df["event_date"])

    # Step 1: Compute event_time for all rows
    df["event_time"] = (df["Date"] - df["event_date"]).dt.days

    # Step 2: Compute CAR_pre from [–10, –1]
    pre_df = (
        df[(df["event_time"] >= pre_window[0]) & (df["event_time"] <= pre_window[1])]
        .groupby(["ticker", "event_date"])["abnormal_return"]
        .sum()
        .reset_index()
    )
    pre_df.rename(columns={"abnormal_return": "CAR_pre"}, inplace=True)

    # Step 3: Compute CAR_post from [0, +10]
    post_df = (
        df[(df["event_time"] >= post_window[0]) & (df["event_time"] <= post_window[1])]
        .groupby(["ticker", "event_date"])["abnormal_return"]
        .sum()
        .reset_index()
    )
    post_df.rename(columns={"abnormal_return": "CAR_post"}, inplace=True)

    # Step 4: Merge CARs back into full df
    df = df.merge(pre_df, on=["ticker", "event_date"], how="left")
    df = df.merge(post_df, on=["ticker", "event_date"], how="left")

    # Step 5: Compute delta_CAR and post indicator
    df["delta_CAR"] = df["CAR_post"] - df["CAR_pre"]
    df["post"] = (df["Date"] >= df["event_date"]).astype(int)

    return df

In [10]:
full_df = compute_car_pre_post_delta_full(tagged_df)

full_df

,Date,ticker,Close,Return,Treatment,expected_return,abnormal_return,event_id,event_date,event_time,CAR_pre,CAR_post,delta_CAR,post
0,2020-12-31,NVDA,13.020754,NaN,1,NaN,<NA>,NaN,NaT,NaN,NaN,NaN,NaN,0
1,2021-01-04,NVDA,13.079099,0.004481,1,NaN,<NA>,NaN,NaT,NaN,NaN,NaN,NaN,0
2,2021-01-05,NVDA,13.369586,0.022210,1,NaN,<NA>,NaN,NaT,NaN,NaN,NaN,NaN,0
3,2021-01-06,NVDA,12.581409,-0.058953,1,NaN,<NA>,NaN,NaT,NaN,NaN,NaN,NaN,0
4,2021-01-07,NVDA,13.308994,0.057830,1,NaN,<NA>,NaN,NaT,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6697,2024-12-23,SP500,5974.069824,0.007287,0,0.000966,0.006321,NaN,NaT,NaN,NaN,NaN,NaN,0
6698,2024-12-24,SP500,6040.040039,0.011043,0,0.001033,0.010009,NaN,NaT,NaN,NaN,NaN,NaN,0
6699,2024-12-26,SP500,6037.589844,-0.000406,0,0.001175,-0.001581,NaN,NaT,NaN,NaN,NaN,NaN,0
6700,2024-12-27,SP500,5970.839844,-0.011056,0,0.001060,-0.012115,NaN,NaT,NaN,NaN,NaN,NaN,0


In [11]:
full_df.to_csv("main_df.csv", index=False)